<h1>Observation Facility Catalogue</h1>

<h3>This is a short notebook to query the Observation Facility catalogue API.</h3>

The catalogue is espose on the SIOS website (https://sios-svalbard.org/sios-ri-catalogue) in a human readable form, and it is also connected to a REST export API which returns a json file containing the information about the Observation Facilities.
This catalogue has been developed to provide an overview of the observation facilities which collect SIOS data. An observation facility can be one instrument or a collection of instruments, e.g. a weather station, and is a term used by the World Meteorological Organization (WMO). The annotation is standardised following the WMO standards as far as possible, in order to make entries unambiguous and interoperable internationally.

This notebook is connecting to the API and displaying the records on a map, including some basic information. More information can be found clicking on the URLs in the popup, or accessing directly the Observation Facility Catalogue.

The updated version of the catalogue includes infrastructures from the Greenland Integrated Observing System (GIOS)

<h3>Description of the endpoints</h3>

- Query the SIOS infrastructures: https://sios-svalbard.org/sios-gios-ri/rest/sios-ri-catalogue.json (Default)
- Query the SIOS infrastructures: https://sios-svalbard.org/sios-gios-ri/rest/sios-ri-catalogue.json?type=1 
- Query the GIOS infrastructures: https://sios-svalbard.org/sios-gios-ri/rest/sios-ri-catalogue.json?type=2 
- Query all infrastructures: https://sios-svalbard.org/sios-gios-ri/rest/sios-ri-catalogue.json?type=All

<h4>Import the necessary modules</h4>

In [1]:
import requests
import pandas as pd
import folium
from folium.plugins import Search
import shapely.wkt

<h4>Fetch only SIOS infrastructure</h4>

In [2]:
def sios_observation_facilities():

    response = requests.get('https://sios-svalbard.org/sios-ri-catalogue/rest/sios-ri-catalogue.json')  
    ofs = []
    ofs_list = []
    for of in response.json():
        if of['title'] not in ofs_list:
            ofs.append({'title':of['title'],
                        'station type':of['OF-Type'], 
                        'geometry': of['OF-Coordinates'], 
                        'station height':of['OF-Height'],
                        'sios-url':of['OF-SIOS-URL'],
                        'institution':of['institution'],
                        'variables':[of['Observed-Variable'].replace('&gt;', ' > ')]})
            ofs_list.append(of['title'])
        else:
            st = next(item for item in ofs if item["title"] == of['title'])
            st['variables'].append(of['Observed-Variable'].replace('&gt;', ' > '))
    return ofs

sios_of = sios_observation_facilities()

sios_m = folium.Map(location=[78, 15], zoom_start=6, tiles='CartoDB positron')

sios_df = pd.DataFrame(sios_of)

for i, r in sios_df.iterrows():
    html = """
    <!DOCTYPE html>
    <html>
    <table style="width:350px;">
        <tr><td colspan="2"><a target='blank' href={url}>{title}</a></td></tr>
        <tr><td style="width:25%">instituion:</td><td>{institution}</td></tr>
        <tr><td>station type:</td><td>{stype}</td></tr>
        <tr><td>geometry:</td><td>{geometry}</td></tr>
        <tr style="border-bottom:1px solid black"><td>height above sea level (m):</td><td>{height}</td></tr>
        <tr><td>variables ({vlen}):</td><td>{variables}</td></tr>
    </table>
    </html> 
    """.format(url=sios_df['sios-url'][i], 
               title=sios_df['title'][i],
               geometry=sios_df['geometry'][i],
               height=sios_df['station height'][i],
               stype=sios_df['station type'][i],
               institution=sios_df['institution'][i],
               variables=sios_df['variables'][i],
               vlen = len(sios_df['variables'][i]))
  
    try:
        el = folium.GeoJson(data=shapely.wkt.loads(r["geometry"]), 
                            tooltip=r['title'])
        el.add_child(folium.Popup(html))
        el.add_to(sios_m)
    except:
        print("Skip invalid geometry", r["geometry"])
    
display(sios_m)

Skip invalid geometry POLYGON ((15.48 77.01, 15.89 77.01, 15.89 76.94, 15.48 76.94))
Skip invalid geometry POLYGON ((11.54 79.05, 12.4 78.91))


In [3]:
sios_df

,title,station type,geometry,station height,sios-url,institution,variables
0,COSMOS - black carbon,land platform (fixed),POINT (11.88611 78.90721),475.00,https://sios-svalbard.org/node/1040,National Institute for Polar Research,[ATMOSPHERE > AEROSOLS > CARBONACEOUS AEROSOLS]
1,Mast for Temperature Gradient,land platform (fixed),POINT (11.83393447 78.92080104),10.00,https://sios-svalbard.org/node/927,Alfred Wegener Institute,[ATMOSPHERE > ATMOSPHERIC TEMPERATURE > SURFAC...
2,Medusa,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60359,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > HALOCARB...
3,Nephlometer,land platform (fixed),POINT (11.88611 78.90721),475.00,https://sios-svalbard.org/node/1007,Stockholm University,[ATMOSPHERE > AEROSOLS > AEROSOL RADIANCE]
4,10-line Global Sampler,land platform (fixed),POINT (11.89418 78.9176),45.00,https://sios-svalbard.org/node/1025,National Institute for Polar Research,[ATMOSPHERE > AEROSOLS > AEROSOL PARTICLE PROP...
...,...,...,...,...,...,...,...
268,Zeppelin Aerodynamic Particle Sizer,land platform (fixed),POINT (11.88934 78.90669),475.00,https://sios-svalbard.org/node/1065,Norwegian Institute for Air Research,[ATMOSPHERE > AEROSOLS > AEROSOL PARTICLE PROP...
269,Zeppelin Europakasse sampler,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60355,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > SULFUR C...
270,Zeppelin Ozone Analyser,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60354,Norwegian Institute for Air Research,[ATMOSPHERE > AIR QUALITY > TROPOSPHERIC OZONE]
271,Zeppelin Picarro,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60360,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > CARBON A...


<h4>Fetch all infrastructure</h4>

In [4]:
def all_observation_facilities():

    response = requests.get('https://sios-svalbard.org/sios-ri-catalogue/rest/sios-ri-catalogue.json?type=All')  
    ofs = []
    ofs_list = []
    for of in response.json():
        if of['title'] not in ofs_list:
            ofs.append({'title':of['title'],
                        'geometry': of['OF-Coordinates'], 
                        'sios-url':of['OF-SIOS-URL'],
                        'institution':of['institution'],
                        'landing-page': of['OF-Landing-Page']
                       })
            ofs_list.append(of['title'])
        else:
            st = next(item for item in ofs if item["title"] == of['title'])
    return ofs

all_of = all_observation_facilities()

all_m = folium.Map(location=[78, 15], zoom_start=2, tiles='CartoDB positron')

all_df = pd.DataFrame(all_of)

for i, r in all_df.iterrows():
    html = """
    <!DOCTYPE html>
    <html>
    <table style="width:350px;">
        <tr><td colspan="2"><a target='blank' href={url}>{title}</a></td></tr>
        <tr><td style="width:25%">instituion:</td><td>{institution}</td></tr>
        <tr><td>geometry:</td><td>{geometry}</td></tr>
    </table>
    </html> 
    """.format(url=all_df['sios-url'][i], 
               title=all_df['title'][i],
               geometry=all_df['geometry'][i],
               institution=all_df['institution'][i],
    )

    if all_df['landing-page'][i] and 'isaaffik' in all_df['landing-page'][i]:
        typeof = 'GIOS'
    else:
        typeof = 'SIOS'

    marker_colors = {'GIOS' : 'red', 
                     'SIOS' : 'blue'}
    try:
        all_el = folium.GeoJson(data=shapely.wkt.loads(r["geometry"]), 
                            tooltip=r['title'],
                            marker=folium.Marker(icon=folium.Icon(color=marker_colors[typeof]))
                           )
        all_el.add_child(folium.Popup(html))
        all_el.add_to(all_m)
    except:
        print("Skip invalid geometry", r["geometry"])
    
display(all_m)

Skip invalid geometry None
Skip invalid geometry POLYGON ((15.48 77.01, 15.89 77.01, 15.89 76.94, 15.48 76.94))
Skip invalid geometry POLYGON ((11.54 79.05, 12.4 78.91))


In [5]:
all_df

,title,geometry,sios-url,institution,landing-page
0,COSMOS - black carbon,POINT (11.88611 78.90721),https://sios-svalbard.org/node/1040,National Institute for Polar Research,None
1,Mast for Temperature Gradient,POINT (11.83393447 78.92080104),https://sios-svalbard.org/node/927,Alfred Wegener Institute,None
2,Medusa,POINT (11.88579 78.90724),https://sios-svalbard.org/node/60359,Norwegian Institute for Air Research,None
3,Nephlometer,POINT (11.88611 78.90721),https://sios-svalbard.org/node/1007,Stockholm University,None
4,10-line Global Sampler,POINT (11.89418 78.9176),https://sios-svalbard.org/node/1025,National Institute for Polar Research,None
...,...,...,...,...,...
928,Zeppelin Ozone Analyser,POINT (11.88579 78.90724),https://sios-svalbard.org/node/60354,Norwegian Institute for Air Research,None
929,Zeppelin Picarro,POINT (11.88579 78.90724),https://sios-svalbard.org/node/60360,Norwegian Institute for Air Research,None
930,thermometers,POINT (11.83108528 78.92163268),https://sios-svalbard.org/node/919,Alfred Wegener Institute,None
931,Åndehullet,POINT (-19.6802 76.66158),https://sios-svalbard.org/node/152565,North-East Greenland Company NANOK [from GIOS],https://isaaffik.org/infrastructures/view/Ånde...
